In [3]:
import datetime
import pandas as pd
import json
import time
import requests as rq
import sys

sys.path = sys.path + ['C:\\Users\\varaw\\Desktop\\Личные проекты\\Untitled Folder\\модули']


import swap
from QuikPy import QuikPy # Работа с QUIK из Python через LUA скрипты QuikSharp

In [2]:
#https://github.com/cia76/QuikPy/blob/master/Examples/05_Stream.py

In [3]:
qp_provider = QuikPy()  # Подключение к локальному запущенному терминалу QUIK по портам по умолчанию
    # qp_provider = QuikPy(host='<Адрес IP>')  # Подключение к удаленному QUIK по портам по умолчанию
    # qp_provider = QuikPy(host='<Адрес IP>', requests_port='<Порт запросов>', callbacks_port='<Порт подписок>')  # Подключение к удаленному QUIK по другим портам

    # Проверяем соединение
print(f'Терминал QUIK подключен к серверу: {qp_provider.IsConnected()["data"] == 1}')
print(f'Отклик QUIK на команду Ping: {qp_provider.Ping()["data"]}')  # Проверка работы скрипта QuikSharp. Должен вернуть Pong

Терминал QUIK подключен к серверу: True
Отклик QUIK на команду Ping: Pong


In [4]:


def make_spread_fut (base,fut): # в формате ["SPBFUT","GZZ4"]
    
    return [float(get_bid_offer(*base,"S")),float(get_bid_offer(*base,"B")),float(get_bid_offer(*fut,"S")),float(get_bid_offer(*fut,"B")),str(datetime.datetime.now())[:-7].replace(":","-")]

def get_fut_info (fut):
    r = rq.get("https://iss.moex.com/iss/engines/futures/markets/forts/boards/RFUD/securities/"+fut+".jsonp?iss.meta=off")
    go = json.loads(r.content)["securities"]["data"][0][14]
    expiration = datetime.datetime.strptime(json.loads(r.content)["securities"]["data"][0][8],"%Y-%m-%d")
    day_to_exp = expiration - datetime.datetime.now()
    day_to_exp = day_to_exp.days+1
    lot = json.loads(r.content)["securities"]["data"][0][13]
    
    base = json.loads(r.content)["securities"]["data"][0][11]
    base_r = rq.get("https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/"+base+".jsonp?iss.meta=off")
    base_lot = json.loads(base_r.content)["securities"]["data"][0][4]                
        
    
    return [go,day_to_exp,[lot,base_lot]]

In [6]:
def gather_trade_information_for_fut (base,fut,time_hour,time_minute):
    global make_spread_fut
    
    df = pd.DataFrame(columns = ["base_S","base_B","fut_S","fut_B","time"])
    df.to_csv(str(base[1]) + "-" + str(fut[1]) + "_" + str(datetime.datetime.now())[:-16] + ".csv")
    
    while datetime.datetime(year = 2023,month = datetime.datetime.now().month,day = datetime.datetime.now().day,hour = time_hour,minute = time_minute) > datetime.datetime.now():
        
        df_temp = pd.DataFrame(data = [make_spread_fut(base,fut)],columns = ["base_S","base_B","fut_S","fut_B","time"])
        df_temp.to_csv(str(base[1]) + "-" + str(fut[1]) + "_" + str(datetime.datetime.now())[:-16] + ".csv", mode='a', header=False,index = False)
        time.sleep(1)

In [7]:
def buy_sell_spread(base,fut,interest,regime,pos,go,day_to_exp,proportion,zapas = 2): #proportion [100,10] 1 фьючерс 100 акций лот 10 акций

    global position,get_fut_info,make_spread_fut,buy_sell
    
    spread_row = make_spread_fut(base,fut)
    need_spread = (spread_row[0]*(proportion[0] + zapas) + go)*interest/365*day_to_exp

    if regime == "B":

        current_spread = spread_row[3] - spread_row[0]*proportion[0]

        print(current_spread,need_spread)

        if current_spread >= need_spread:

            buy_sell(base,"M","B",qty = int(proportion[0]/proportion[1]))
            buy_sell(fut,"M","S")
            
            if pos == "M":
                position += spread_row[0]*(proportion[0] + zapas) + go
            else:
                position += 1
                
    if regime == "S":

        current_spread = spread_row[2] - spread_row[1]*proportion[0]

        print(current_spread,need_spread)

        if current_spread <= need_spread:

            buy_sell(base,"M","S",qty = int(proportion[0]/proportion[1]))
            buy_sell(fut,"M","B")
            
            if pos == "M":
                position += spread_row[0]*(proportion[0] + zapas) + go
            else:
                position += 1

In [21]:
need_position = 20
inter = 0.175
position = 0
futures = "ALZ3"
base_activ = "ALRS"


go,day_to_exp,proportion = get_fut_info(futures)

while need_position > position:

    buy_sell_spread(["TQBR",base_activ],["SPBFUT",futures],inter,"S","A",go,day_to_exp,proportion)
    time.sleep(0.2)

122.99999999999909 127.92135616438355
Новая лимитная/рыночная заявка отправлена на рынок: True 1079
Новая лимитная/рыночная заявка отправлена на рынок: True 1080
124.0 127.92135616438355
Новая лимитная/рыночная заявка отправлена на рынок: True 1081
Новая лимитная/рыночная заявка отправлена на рынок: True 1082
126.0 127.92135616438355
Новая лимитная/рыночная заявка отправлена на рынок: True 1083
Новая лимитная/рыночная заявка отправлена на рынок: True 1084
125.99999999999909 127.96536986301369
Новая лимитная/рыночная заявка отправлена на рынок: True 1085
Новая лимитная/рыночная заявка отправлена на рынок: True 1086
125.0 127.93602739726025
Новая лимитная/рыночная заявка отправлена на рынок: True 1087
Новая лимитная/рыночная заявка отправлена на рынок: True 1088
127.0 127.95069863013696
Новая лимитная/рыночная заявка отправлена на рынок: True 1089
Новая лимитная/рыночная заявка отправлена на рынок: True 1090
127.0 127.95069863013696
Новая лимитная/рыночная заявка отправлена на рынок: Tru

In [8]:
def spread_speculation(position_money,spread,max_min,position_activ_b = 0,position_activ_s = 0): #spread ["FUTSPREAD","EuZ3EuH4"] max_min [1250,1200,1300]

    while True:
        temp_s = get_bid_offer(*spread,"S")
        
        if (temp_s <= max_min[1]) & (position_money > 0):
            
            buy_sell(spread,"M","B",qty = 1,price = -1)
            position_activ_b += 1
            position_money -= 1
            
        if (temp_s <= max_min[0]) & (position_activ_s > 0):
            
            buy_sell(spread,"M","B",qty = 1,price = -1)
            position_activ_s -= 1
            position_money += 1
        
        temp_b = get_bid_offer(*spread,"B")
        
        if (temp_b >= max_min[0]) & (position_activ_b > 0):
            
            buy_sell(spread,"M","S",qty = 1,price = -1)
            position_activ_b -= 1
            position_money += 1
            
        if (temp_b >= max_min[2]) & (position_money > 0):
            
            buy_sell(spread,"M","S",qty = 1,price = -1)
            position_activ_s += 1
            position_money -= 1
        

        time.sleep(0.05)
        
        if datetime.datetime.now() > datetime.datetime(year = 2023,month = datetime.datetime.now().month,day = datetime.datetime.now().day,hour = 23,minute = 45):
            print("ночь")
            time.sleep(560*60)
        
        if datetime.datetime(year = 2023,month = datetime.datetime.now().month,day = datetime.datetime.now().day,hour = 19,minute = 6) > datetime.datetime.now() > datetime.datetime(year = 2023,month = datetime.datetime.now().month,day = datetime.datetime.now().day,hour = 18,minute = 49):
            print("вечерний клиринг")
            time.sleep(17*60)
            
        if datetime.datetime(year = 2023,month = datetime.datetime.now().month,day = datetime.datetime.now().day,hour = 14,minute = 6) > datetime.datetime.now() > datetime.datetime(year = 2023,month = datetime.datetime.now().month,day = datetime.datetime.now().day,hour = 13,minute = 59):
            print("дневной клиринг")
            time.sleep(7*60)
            
        
        

In [14]:
spread_speculation(0,["FUTSPREAD","EuZ3EuH4"],[1600,1225,2045],390,0)

KeyboardInterrupt: 